# 0. Imports, funcs

In [159]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
from cancer_nutrition.utils.db.frontend import DBProvider
from cancer_nutrition.utils.constants.db import *
from cancer_nutrition.utils.form.form import form_to_dict

In [3]:
db_provider = DBProvider(DB_NAME, DB_USER, DB_PASSWD, DB_HOST, 11459)

In [158]:
from cancer_nutrition.utils.value import Value
from cancer_nutrition.utils.age import Age

from cancer_nutrition.nutrition.parenteral_selection import ParenteralSelector
from cancer_nutrition.nutrition.main import NutritionRecommender, RecommendationSet
from cancer_nutrition.utils.measurement.nutrition_type_fraction import NutritionTypeFractions
from cancer_nutrition.utils.measurement.perfect_mass import PerfectMass

from cancer_nutrition.nutrition.protein import ProteinDerivation
from cancer_nutrition.nutrition.lipid import LipidDerivation
from cancer_nutrition.nutrition.carbohydrate import CarbohydrateDerivation
from cancer_nutrition.nutrition.energy import EnergyDerivation


In [155]:
def get_rec_set(data_dict):
    classes = {"Белок": ProteinDerivation,
               "Жиры": LipidDerivation,
               "Углеводы": CarbohydrateDerivation,
#                "ККал": EnergyDerivation
              }
    due_demands = {k + "_due": _class.compute(data_dict, due_only=True)
                   for k, _class in classes.items()}
#     data_dict[RefeedingRisk.name] = RefeedingRisk.compute(**data_dict, **due_demands)
#     data_dict["Производные параметры"] = data_dict["Производные параметры"] + [RefeedingRisk.name]

    demands = {k: dict(zip(["value", "limits", "due"],
                           _class.compute(data_dict)))
               for k, _class in classes.items()}

#     print(demands, due_demands)

    fractions = NutritionTypeFractions.compute(demands, **data_dict)
    data_dict["fractions"] = fractions

#     data_dict[EnteralTagging.name] = EnteralTagging.compute(**data_dict)

    out_demands, out_parenterals = ParenteralSelector.compute(data_dict, demands, data_dict["ПЭП"])
    # print(out_parenterals, out_demands)

#     enteral_tags = self.db_wrapper.get_nutrition_tags(enteral=True)
#     out_en_demands, out_enterals = {}, {}

#     en_due_demands = {"ККал": due_demands["ККал_due"],
#                       "Белок": EnteralSelector._p_en_f(data_dict["Возраст"]["лет"],
#                                                        due_demands["ККал_due"]),
#                       "Жиры": EnteralSelector._l_en_f(data_dict["Возраст"]["лет"],
#                                                       due_demands["ККал_due"]),
#                       "Углеводы": EnteralSelector._c_en_f(data_dict["Возраст"]["лет"],
#                                                           due_demands["ККал_due"])
#                       }

#     for sip in [False, True]:
#         if data_dict["fractions"]["сипинг" if sip else "зонд"] == 0:
#             continue
#         en_demands, enterals = EnteralSelector.compute(data_dict,
#                                                        demands,
#                                                        data_dict["ЭП"],
#                                                        enteral_tags,
#                                                        sipping=sip)
#         for k in enterals:
#             enterals[k] = {"dose": round(enterals[k] * data_dict["fractions"]["сипинг" if sip else "зонд"] / 10) * 10,
#                            "tags": {tag: int(tag in enteral_tags[k])
#                                     for tag in data_dict[EnteralTagging.name].keys()},
#                            "features": {fea: val for fea, val in data_dict["ЭП"][k].items()}}
#         out_en_demands[f"sipping = {sip}"] = en_due_demands  # en_demands  # {k: v / data_dict["Масса тела"]["кг"] for k,v in en_demands.items()}
#         out_enterals[f"sipping = {sip}"] = enterals

    recom_set = RecommendationSet(data_dict)
    out_parenterals = {k: {"dose": v,
                           "features": data_dict["ПЭП"][k]}
                       for k,v in out_parenterals.items()}

    recom_set["ПЭП"] = {"nutrition": out_parenterals,
                        "demands": {k + "_due": dem["due"]
                                    for k, dem in demands.items()}}
#     for sip in [False, True]:
#         recom_set["сипинг" if sip else "зонд"] = {"nutrition": out_enterals.get(f"sipping = {sip}", {}),
#                                                   "demands": out_en_demands.get(f"sipping = {sip}", {})}

    return recom_set

# 1. Parameter combinations

In [131]:
domains = {'Возраст':[Age({"мес": 12}), Age({"мес": 34}), Age({"мес": 72})],
           'Масса тела': [Value(5, "кг"), Value(15, "кг"), Value(36, "кг"), Value(48, "кг")],
           "Пол": ["Мужской", "Женский"],
            'Нормальный нутритивный статус': [0,1],
            'Легкая белково-энергетическая недостаточность': [0,1],
            'Умеренная белково-энергетическая недостаточность': [0,1],
            'Тяжелая белково-энергетическая недостаточность': [0,1],
            'Избыточная масса тела': [0,1],
            'Ожирение 1 степени': [0,1],
           'Ожирение 2 степени': [0,1],
           'Ожирение 3 степени': [0,1],
           'Ожирение тяжелое': [0,1],
            'Дефицит висцерального пула белка': [0,1],
            'Тек*Небелковые ккал': [19, 26],
            'Острая печеночная недостаточность': [0,1],
            'Лабораторные признаки холестаза': [0,1],
            'Острое повреждение почек': [0,1],
            'Лабораторные признаки острого панкреатита': [0,1],
           "Хроническая болезнь почек": [0,1],
           "Потребность в дотации кислорода": [0,1],
           "Наличие центрального венозного катетера (либо будет установлен в ближайшие 48 часов)": [0,1],
            'Ожидаемая токсичность': [0,1],
           "Этап терапии": ["CAR T-Cells терапия", "другое"],
            'Риск рефидинг-синдрома': np.arange(0, 1.1, 0.1).tolist(),
            'Триглицериды': [1.0, 3.1, 4.6], 
           "Активность пациента": ["активен и мобилен", 
                                   "активность ограничена", 
                                   "тяжелое состояние - требует поддержки жизненных функций"]
          }

In [132]:
nutr_status_keys = list(domains)[3:12]

In [133]:
nutr_status_combs = np.vstack(np.meshgrid(*[domains[k] 
                                for k in nutr_status_keys])).reshape((len(nutr_status_keys), -1)).T
nutr_status_combs = nutr_status_combs[nutr_status_combs.sum(axis=1) == 1]

In [134]:
combs = np.vstack(np.meshgrid(*[domains[k] 
                                for k in domains 
                                if k not in nutr_status_keys])).reshape((len(domains) - len(nutr_status_keys), 
                                                                         -1)).T

In [135]:
df_combs = pd.DataFrame(combs, 
                        columns=[k for k in domains if k not in nutr_status_keys])

In [136]:
parenterals = db_provider.get_nutrition_features(False)

In [148]:
constants = {"Cколько часов в сутках доступно для потенциального ПЭП": 24, 
             "Частичная нутритивная поддержка": 0,
             "Полное восполнение питания": 1,
             "Доля энтерального питания (сипинг)": 0,
             "Доля энтерального питания (зонд/стома)": 0,
             "Доля парентерального питания": 100, 
             "ПЭП": parenterals, 
             "Производные параметры": []}

In [149]:
analyses = {"Глюкоза": [4, 12], 
            "Лактат": [1.5, 2.5]}

In [150]:
analysis_combs = np.vstack(np.meshgrid(*list(analyses.values()))).reshape((len(analyses), -1)).T

In [151]:
def check_combination(**comb):
    try:
        rec_set = get_rec_set(comb)
        return True, rec_set
    except Exception as e:
        return False, e
        

In [157]:
len(nutr_status_keys) * len(analysis_combs) * len(df_combs)

175177728

In [161]:
for act_nutr_status in nutr_status_keys:
    nutr_status = {k: 0 for k in nutr_status_keys}
    nutr_status[act_nutr_status] = 1
    for an_comb in analysis_combs:
        analysis_comb = {item: an_comb[i] 
                         for i, item in enumerate(list(analyses.keys()))}
        for i in tqdm(range(len(df_combs))):
            perfect_mass = PerfectMass.compute(**nutr_status)
            comb_res = check_combination(**df_combs.loc[i].to_dict(), 
                                         **constants, 
                                         **nutr_status, 
                                         **analysis_comb, 
                                         **{"Идеальная масса тела": perfect_mass})
            
#             if not comb_res[0]:
#                 raise comb_res[1]

#             break
#         break
#     break


  0%|          | 0/4866048 [00:00<?, ?it/s]


KeyboardInterrupt

